# Fine-Tuning Simple Policy Model

## Dependencies and Keys

In [1]:
!pip install -qU bitsandbytes datasets accelerate loralib peft transformers trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 10.9 MB/s eta 0:00:00


## Download the Data

In [2]:
!pip install -qU gdown

Load input data

In [3]:
!gdown 1b53PKIEg7-22HXa2sa8AJlu5eP5XEqX0

Downloading...
From: https://drive.google.com/uc?id=1b53PKIEg7-22HXa2sa8AJlu5eP5XEqX0
To: /content/inputs.zip
100% 182k/182k [00:00<00:00, 5.54MB/s]


Load output data

In [4]:
!gdown 1EObDzL4wc1BDNNRWvA2ucfc9r9ySvCE6

Downloading...
From: https://drive.google.com/uc?id=1EObDzL4wc1BDNNRWvA2ucfc9r9ySvCE6
To: /content/outputs2.zip
100% 125k/125k [00:00<00:00, 4.88MB/s]


Unzipping files...

In [5]:
!unzip "inputs.zip"

Archive:  inputs.zip
   creating: inputs/
  inflating: __MACOSX/._inputs       
  inflating: inputs/policy014.md     
  inflating: __MACOSX/inputs/._policy014.md  
  inflating: inputs/policy012-2.md   
  inflating: __MACOSX/inputs/._policy012-2.md  
  inflating: inputs/policy011-5.md   
  inflating: __MACOSX/inputs/._policy011-5.md  
  inflating: inputs/policy007-5.md   
  inflating: __MACOSX/inputs/._policy007-5.md  
  inflating: inputs/policy004-2.md   
  inflating: __MACOSX/inputs/._policy004-2.md  
  inflating: inputs/policy005-3.md   
  inflating: __MACOSX/inputs/._policy005-3.md  
  inflating: inputs/policy006-4.md   
  inflating: __MACOSX/inputs/._policy006-4.md  
  inflating: inputs/policy010-4.md   
  inflating: __MACOSX/inputs/._policy010-4.md  
  inflating: inputs/policy013-3.md   
  inflating: __MACOSX/inputs/._policy013-3.md  
  inflating: inputs/policy004.md     
  inflating: __MACOSX/inputs/._policy004.md  
  inflating: inputs/policy011-1.md   
  inflating: __MACOSX/inpu

In [6]:
!unzip -u "outputs2.zip"

Archive:  outputs2.zip
   creating: outputs2/
  inflating: __MACOSX/._outputs2     
  inflating: outputs2/simplified-policy014-2.md  
  inflating: __MACOSX/outputs2/._simplified-policy014-2.md  
  inflating: outputs2/simplified-policy001-5.md  
  inflating: __MACOSX/outputs2/._simplified-policy001-5.md  
  inflating: outputs2/simplified-policy002-2.md  
  inflating: __MACOSX/outputs2/._simplified-policy002-2.md  
  inflating: outputs2/simplified-policy003-3.md  
  inflating: __MACOSX/outputs2/._simplified-policy003-3.md  
  inflating: outputs2/simplified-policy001-1.md  
  inflating: __MACOSX/outputs2/._simplified-policy001-1.md  
  inflating: outputs2/simplified-policy008.md  
  inflating: __MACOSX/outputs2/._simplified-policy008.md  
  inflating: outputs2/simplified-policy009.md  
  inflating: __MACOSX/outputs2/._simplified-policy009.md  
  inflating: outputs2/simplified-policy014-3.md  
  inflating: __MACOSX/outputs2/._simplified-policy014-3.md  
  inflating: outputs2/simplified-pol

Let's conver the data into a json array format.

In [7]:
import os
import json

INPUTS_DIR = "./inputs"
OUTPUTS_DIR = "./outputs2"

json_array = []

for input_file in os.listdir(INPUTS_DIR):
    if input_file.endswith(".md"):
        base_name = input_file.split(".")[0]
        output_file = f"simplified-{base_name}.md"

        input_path = os.path.join(INPUTS_DIR, input_file)
        output_path = os.path.join(OUTPUTS_DIR, output_file)

        if os.path.exists(output_path):
            with open(input_path, "r", encoding="utf-8") as f:
                input_content = f.read()

            with open(output_path, "r", encoding="utf-8") as f:
                output_content = f.read()

            json_array.append({
                "input": input_content,
                "output": output_content,
                "id": base_name
            })

Now we can convert the data into a HuggingFace Dataset

In [8]:
from datasets import Dataset, load_dataset

policy_dataset = Dataset.from_list(json_array); policy_dataset

Dataset({
    features: ['input', 'output', 'id'],
    num_rows: 84
})

Let's see what the original input and output look like.

In [9]:
print(policy_dataset[0]['input'])

Base Policy:
# ACORD

DATE (MM/DD/YYYY): 7/30/2022

THIS CERTIFICATE OF LIABILITY INSURANCE

THIS CERTIFICATE IS ISSUED AS A MATTER OF INFORMATION ONLY AND CONFERS NO RIGHTS UPON THE CERTIFICATE HOLDER. THIS CERTIFICATE DOES NOT AFFIRMATIVELY OR NEGATIVELY AMEND, EXTEND OR ALTER THE COVERAGE AFFORDED BY THE POLICIES BELOW. THIS CERTIFICATE OF INSURANCE DOES NOT CONSTITUTE A CONTRACT BETWEEN THE ISSUING INSURER(S), AUTHORIZED REPRESENTATIVE OR PRODUCER, AND THE CERTIFICATE HOLDER.

IMPORTANT: If the certificate holder is an ADDITIONAL INSURED, the policy(ies) must be endorsed. If SUBROGATION IS WAIVED, subject to the terms and conditions of the policy, certain policies may require an endorsement. A statement on this certificate does not confer rights to the certificate holder in lieu of such endorsement(s).

# PRODUCER

Trust Partners

CONTACT

NAME:

PHONE: 888-555-1234

FAX (A/C, No): 888-555-4321

PO BOX 10127

E-MAIL ADDRESS: certificates@trustpartners.com

Nashville TN 37201

INSUR

In [10]:
print(policy_dataset[0]['output'])

Congratulations! You got yourself a Commercial General Liability Policy for your business, LT CARGO INC, at 1025 N Broadway St Ste 1020, Los Angeles CA 90012. Your policy number is **GTUZ000982-02**.

We want to make sure you know what you're getting for your premium, so we did our best to make this policy short and sweet.

Please take a few minutes to read through, and let us know if you have any questions. You can always change coverages, and more.

## Who's covered?
This policy covers **LT CARGO INC**.

## When?
This policy covers events that started after May 11, 2022, and before May 11, 2023.

## Against what?
We protect you against liabilities from accidents, injuries, and damages that occur in the course of your business operations. There are important limitations, though, so please read on.

## For how much?
We provide coverage up to a certain limit. Here is a quick overview of the limits you chose (and can change):

- Each Occurrence: $1,500,000
- Damage to Rented Premises (Ea

In [11]:
# train-test split
prepared_policy_dataset = policy_dataset.train_test_split(test_size=0.1); prepared_policy_dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'id'],
        num_rows: 75
    })
    test: Dataset({
        features: ['input', 'output', 'id'],
        num_rows: 9
    })
})

## Fine-Tuning Prep

### Defining Instruction Prompt

In [12]:
# define the fine-tuning prompts
INSTRUCTION_PROMPT_TEMPLATE = """\
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Please translate the following insurance policy into a simplified format<|eot_id|><|start_header_id|>user|end_header_id|>

[INSURANCE_POLICY]{INPUT}[END_INSURANCE_POLICY]<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

RESPONSE_TEMPLATE = """\
{OUTPUT}<|eot_id|><|end_of_text|>
"""

In [13]:
# helper function that converts a dataset row into the prompt
def create_instruction(sample: str, return_response: bool = True) -> str:
    prompt = INSTRUCTION_PROMPT_TEMPLATE.format(INPUT=sample['input'])

    if return_response:
        prompt += RESPONSE_TEMPLATE.format(OUTPUT=sample['output'])

    return prompt

In [14]:
print(create_instruction(policy_dataset[0]))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Please translate the following insurance policy into a simplified format<|eot_id|><|start_header_id|>user|end_header_id|>

[INSURANCE_POLICY]Base Policy:
# ACORD

DATE (MM/DD/YYYY): 7/30/2022

THIS CERTIFICATE OF LIABILITY INSURANCE

THIS CERTIFICATE IS ISSUED AS A MATTER OF INFORMATION ONLY AND CONFERS NO RIGHTS UPON THE CERTIFICATE HOLDER. THIS CERTIFICATE DOES NOT AFFIRMATIVELY OR NEGATIVELY AMEND, EXTEND OR ALTER THE COVERAGE AFFORDED BY THE POLICIES BELOW. THIS CERTIFICATE OF INSURANCE DOES NOT CONSTITUTE A CONTRACT BETWEEN THE ISSUING INSURER(S), AUTHORIZED REPRESENTATIVE OR PRODUCER, AND THE CERTIFICATE HOLDER.

IMPORTANT: If the certificate holder is an ADDITIONAL INSURED, the policy(ies) must be endorsed. If SUBROGATION IS WAIVED, subject to the terms and conditions of the policy, certain policies may require an endorsement. A statement on this certificate does not confer rights to the certificate holder in lieu of s

### Loading the Model and Tokenizer

In [15]:
# log into huggingface_hub
from huggingface_hub import notebook_login

notebook_login()

In [16]:
# load the model
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_id = "NousResearch/Meta-Llama-3-8B-Instruct"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    quantization_config=quantization_config
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [17]:
# load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [18]:
# setup Lora config
from peft import LoraConfig

peft_config = LoraConfig(
    r=32,
    lora_alpha=64,
    bias="none",
    target_modules="all-linear",
    task_type="CAUSAL_LM"
)

## Fine-Tuning `Meta-LLama3-8B-Instruct`

In [19]:
# define the training arguments
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="clearpolicy-llama-3-8binstruct-v2",
    num_train_epochs=4,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="adamw_torch_fused",
    save_strategy="epoch",
    learning_rate=2e-4,
    fp16=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    push_to_hub=True
)

In [20]:
# setup SFTTrainer
from trl import SFTTrainer

max_seq_length=4096

trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=prepared_policy_dataset['train'],
    eval_dataset=prepared_policy_dataset['test'],
    formatting_func=create_instruction,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    dataset_kwargs={
        "add_special_tokens": False,
        "append_concat_token": False
    }
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:440: UserWarning: The passed formatting_func has more than one argument. Usually that function should have a single argument `example` which corresponds to the dictionary returned by each element of the dataset. Make sure you know what you are doing.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [21]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=T

TrainOutput(global_step=28, training_loss=0.6011050088065011, metrics={'train_runtime': 222.0773, 'train_samples_per_second': 0.486, 'train_steps_per_second': 0.126, 'total_flos': 2.0142282208444416e+16, 'train_loss': 0.6011050088065011, 'epoch': 4.0})

In [22]:
trainer.save_model()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Let's clean up the memory...

In [23]:
del model
del trainer
torch.cuda.empty_cache()

Ok, so because we're using LoRA, we need to load the PEFT model and them merge the LoRA layers back into the original model so we can then use the Hugging Face pipeline.

In [24]:
from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(
    args.output_dir,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [25]:
merged_model = model.merge_and_unload()

## Testing the Fine-tuned Model

In [26]:
# create hf pipeline
from transformers import pipeline

simplify_pipe = pipeline(
    "text-generation",
    model=merged_model,
    tokenizer=tokenizer,
    max_new_tokens=1000,
    return_full_text=False
)

In [27]:
# original input
sample = prepared_policy_dataset['test'][0]
print(sample['input'])

# Lemonade

5 Crosby St., New York, NY 10013

# RENTERS POLICY DECLARATIONS

|POLICY NUMBER|POLICY START DATE|POLICY EXPIRES ON|
|---|---|---|
|PX748209T129|August 20, 2023 10:30 AM|August 22, 2024 11:30 AM|

# INSURED PROPERTY ADDRESS

NAME OF INSURED: RYAN STEVENSON

1230 Peachtree St,

Atlanta, GA 30309

INSURED EMAIL: ryanstevenson1@gmail.com

# COVERAGE SUMMARY

|COVERAGE|MAXIMUM AMOUNT|COST|
|---|---|---|
|Personal Property|$50,000|Included|
|Loss Of Use|$15,000|Included|
|Personal Liability|$120,000|Included|
|Medical Payments To Others|$2,000|Included|
|Portable Electronics|$5,000|Included|
|$300 Deductible|Included| |

Total Premium $250 ($20.83/mo)

© Copyright 2017 by LEMONADE INSURANCE COMPANY (a stock company) I 5 Crosby St., New York, NY 10013 I help@lemonade.com I LEM-DEC-01-TX
---
|FOR|AMOUNT|
|---|---|
|Fire Protection|$5.00|
|Local Fire Alarm| |

Total Discount: $5.00 (Annual)

© Copyright 2017 by LEMONADE INSURANCE COMPANY (a stock company) 15 Crosby St., New York, N

In [28]:
# generate output
instruction = create_instruction(sample, return_response=False)
outputs = simplify_pipe(instruction, do_sample=True, max_new_tokens=512)

In [29]:
print(outputs[0]['generated_text'])

Congratulations! You got yourself a Renters Policy for your property at 1230 Peachtree St, Atlanta, GA 30309. Your policy number is **PX748209T129**.

We want to make sure you know what you're getting for your **$250 per year ($20.83/month)**, so we did our best to make this policy short and sweet.

Please take a few minutes to read through, and let us know if you have any questions. You can always change coverages, and more.

## Who's covered?
This policy covers **Ryan Stevenson and his belongings**.

## When?
This policy covers events that started after August 20, 2023 10:30 AM, and before August 22, 2024 11:30 AM.

## Against what?
We protect you against loss or damage to personal property, loss of use, personal liability, medical payments to others, and loss or damage to portable electronics. There are important limitations, though, so please read on.

## For how much?
We provide coverage up to a certain limit. Here is a quick overview of the limits you chose (and can change):

- P

Great! Another example now

In [30]:
# original input
sample = prepared_policy_dataset['test'][8]
print(sample['input'])

CERTIFICATE OF PET LIABILITY INSURANCE

THIS CERTIFICATE IS ISSUED AS A MATTER OF INFORMATION ONLY AND CONFERS NO RIGHTS UPON THE CERTIFICATE HOLDER. THIS CERTIFICATE DOES NOT AFFIRMATIVELY OR NEGATIVELY AMEND, EXTEND OR ALTER THE COVERAGE AFFORDED BY THE POLICIES BELOW. THIS CERTIFICATE OF INSURANCE DOES NOT CONSTITUTE A CONTRACT BETWEEN THE ISSUING INSURER(S), AUTHORIZED REPRESENTATIVE OR PRODUCER, AND THE CERTIFICATE HOLDER.

IMPORTANT: If the certificate holder is an ADDITIONAL INSURED, the policy(ies) must be endorsed. If SUBROGATION IS WAIVED, subject to the terms and conditions of the policy, certain policies may require an endorsement. A statement on this certificate does not confer rights to the certificate holder in lieu of such endorsement(s).

|INSURER|CONTACT|
|---|---|
|AMERICAN PET INSURANCE COMPANY|NAME: AMERICAN PET INSURANCE CO|
| |TEXAS USA|
|INSURERS AFFORDING COVERAGE|NAIC #|
|INSURER A APIC TEXAS|0143|
|INSURED|INSURER B|
|CHARLIE (12 WEEKS OLD MALE BERNESE MOUNTA

In [31]:
# generate output
instruction = create_instruction(sample, return_response=False)
outputs = simplify_pipe(instruction, do_sample=True, max_new_tokens=512)

In [32]:
print(outputs[0]['generated_text'])

Congratulations! You got yourself a Pet Liability Insurance Policy for your 12-week-old male Bernese Mountain Puppy, Charlie. Your policy number is **PTL01574**. We want to make sure you know what you're getting for your premium, so we did our best to make this policy short and sweet.

Please take a few minutes to read through, and let us know if you have any questions. You can always change coverages, and more.

## Who's covered?
This policy covers **Charlie, your Bernese Mountain Puppy**.

## When?
This policy covers events that started after August 18, 2021, and before September 18, 2021.

## Against what?
We protect you against costs related to injuries or property damage caused by Charlie. There are important limitations, though, so please read on.

## For how much?
We provide coverage up to a certain limit. Here is a quick overview of the limits you chose (and can change):

- Bodily Injury: $5000.00
- Property: $5000.00

These amounts indicate the maximum we will reimburse you, i